In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
# Imports
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1.  AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


spark.sparkContext.addFile(url)
HomeSalesData = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show DataFrame
HomeSalesData.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Temporary view of the DataFrame.

HomeSalesData.createOrReplaceTempView('home_sales')

In [5]:
# Schema information
HomeSalesData.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

Avg_Price_4bedroom = spark.sql("""SELECT
                                YEAR(date) as `Year Sold`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  home_sales
                                where
                                  bedrooms = 4
                                group by
                                  YEAR(date)
                                ORDER BY
                                  YEAR(date) desc
                                  """)

Avg_Price_4bedroom.show()

+---------+---------+
|Year Sold|AVG Price|
+---------+---------+
|     2022|296363.88|
|     2021|301819.44|
|     2020|298353.78|
|     2019| 300263.7|
+---------+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
Avg_Price_3bed3bath = spark.sql("""SELECT
                                date_built as `Year Built`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  home_sales
                                where
                                  bedrooms = 3 and bathrooms = 3
                                group by
                                  date_built
                                ORDER BY
                                  date_built desc
                                  """)

Avg_Price_3bed3bath.show()

+----------+---------+
|Year Built|AVG Price|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [8]:
# 5. What is the average price of a home for each year the home was built,that have 3 bedrooms, 3 bathrooms, with two floors,and are greater than or equal to 2,000 square feet, rounded to two decimal places?
Avg_Price_3bed3bath2floors = spark.sql("""SELECT
                                              date_built as `Year Built`,
                                              ROUND(AVG(price),2) as `AVG Price`
                                              from
                                                home_sales
                                              where
                                                bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
                                              group by
                                                date_built
                                              ORDER BY
                                                date_built desc
                                                """)

Avg_Price_3bed3bath2floors.show()


+----------+---------+
|Year Built|AVG Price|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,having an average home price greater than or equal to $350,000? Order by descending view rating.Although this is a small dataset, determine the run time for this query.

start_time = time.time()
Avg_Price_view = spark.sql("""SELECT
                                view as `View Rating`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  home_sales
                                group by
                                  view
                                having
                                AVG(price) >= 350000
                                order by
                                  view desc
                                    """)

Avg_Price_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|View Rating| AVG Price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.9474477767944336 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Calculates,the average price of a home per "view" rating,having an average home price greater than or equal to $350,000.

start_time = time.time()
Avg_Price_View = spark.sql("""SELECT
                                view as `View Rating`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  home_sales
                                group by
                                  view
                                having
                                AVG(price) >= 350000
                                order by
                                  view desc
                                    """)

Avg_Price_View.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|View Rating| AVG Price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.5405788421630859 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
HomeSalesData.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [14]:
# 11. Read the parquet formatted data.
P_HomeSalesData_p=spark.read.parquet('home_sales_partitioned')

In [15]:
# 12. Temporary table for the parquet data 'p_home_sales_p'.
P_HomeSalesData_p.createOrReplaceTempView('p_home_sales_p')

In [16]:
# 13.calculates the average price of a home per "view" rating, having an average home price greater than or equal to $350,000.Determine

start_time = time.time()
Avg_Price_View = spark.sql("""SELECT
                                view as `View Rating`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  p_home_sales_p
                                group by
                                  view
                                having
                                AVG(price) >= 350000
                                order by
                                  view desc
                                    """)

Avg_Price_View.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|View Rating| AVG Price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.9160075187683105 seconds ---


In [17]:
# 14. Partition by the "view" field on the formatted parquet "home_sales_partitioned2"
HomeSalesData.write.partitionBy("view").mode("overwrite").parquet("home_sales_partitioned2")

In [18]:
# 15. Read the parquet formatted data ('home_sales_partitioned2').
p_HomeSalesData_p2=spark.read.parquet('home_sales_partitioned2')


In [19]:
 # Temporary table for the parquet data 'p_home_sales_p2'.
 p_HomeSalesData_p2.createOrReplaceTempView('p_home_sales_p2')

In [20]:
# calculates:
# the average price of a home per "view" rating
# having an average home price greater than or equal to $350,000.
start_time = time.time()

Avg_Price_View = spark.sql("""SELECT
                                view as `View Rating`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  p_home_sales_p2
                                group by
                                  view
                                having
                                AVG(price) >= 350000
                                order by
                                  view desc
                                    """)

Avg_Price_View.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|View Rating| AVG Price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 1.304619312286377 seconds ---


In [21]:
# 17. calculate the average price of home
start_time = time.time()
Avg_Price_View = spark.sql("""SELECT
                                view as `View Rating`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  home_sales
                                where
                                  view = 100
                                group by
                                  view
                               """)

Avg_Price_View.show()
print("--- %s seconds ---" % (time.time() - start_time))

+-----------+---------+
|View Rating|AVG Price|
+-----------+---------+
|        100|1026669.5|
+-----------+---------+

--- 0.5001888275146484 seconds ---


In [22]:
# 18. Uncache the home_sales temporary table.

spark.sql("uncache table home_sales")

DataFrame[]

In [23]:
# 15. home_sales is no longer cached
spark.catalog.isCached("home_sales")


False

In [24]:
#calculate the average price of homes with a view rating of 100

start_time = time.time()
Avg_Price_View = spark.sql("""SELECT
                                view as `View Rating`,
                                ROUND(AVG(price),2) as `AVG Price`
                                from
                                  home_sales
                                where
                                  view = 100
                                group by
                                  view
                               """)

Avg_Price_View.show()
print("--- %s seconds ---" % (time.time() - start_time))

+-----------+---------+
|View Rating|AVG Price|
+-----------+---------+
|        100|1026669.5|
+-----------+---------+

--- 0.5092318058013916 seconds ---
